# Ensemble Classifiers: Reduced Featureset
---
Reduced featureset (18 features): ['illday', 'rash', 'redeyes', 'redplt', 'clnode',
		'redhands', 'pwbc', 'ppolys', 'pbands', 'plymphs', 'pmonos',
		'peos', 'pesr', 'pcrp', 'pplts', 'palt', 'pggt', 'zhemo']

### Pre-Training Setup (Load Reduced-Features Data)

In [1]:
import sys
sys.path.append('../') # Make parent folder visible
import numpy as np
from scipy.stats import randint

from sklearn import svm, linear_model, neighbors, ensemble, naive_bayes, \
    neural_network, tree, gaussian_process, discriminant_analysis
    
import matplotlib.pyplot as plt

from preprocess import load_data
from model_helpers.models import compute_confusion, explain_confusion, get_fc_kd_thresholds, \
    test_model, plot_cv_roc_curves, ScikitModel

In [ ]:
x, y, ids = load_data.load_expanded(one_hot=False, fill_mode='mean', reduced_features=True)

### Test out Candidate Learners

In [ ]:
# SVM
svm_model = ScikitModel(svm.SVC(probability=True), 
                       {
                           'C': np.logspace(-3, 3, 100),
                           'gamma': np.logspace(-3, 3, 100),
                           'kernel': ['linear', 'rbf', 'poly']
                       },
                       random_search=True,
                       n_iter=100,
                       scoring='roc_auc',
                       verbose=True)
svm_rocs = test_model(svm_model,
                      x, y,
                      allow_indeterminates=True)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [ ]:
# Logistic Regression
lr_model = ScikitModel(linear_model.LogisticRegression(), 
                       params={
                           'C': np.logspace(-2, 2, 5)
                       },
                       random_search=False,
                       scoring='roc_auc',
                       verbose=True)
lr_rocs = test_model(lr_model,
                     x, y,
                     allow_indeterminates=True)

In [ ]:
plot_cv_roc_curves(lr_rocs)

In [ ]:
# Random Forest
rf_model = ScikitModel(ensemble.RandomForestClassifier(), 
                       params={
                           'n_estimators': randint(10, 500),
                           'max_features': randint(3, 15),
                           'min_samples_split': randint(2, 50),
                           'min_samples_leaf': randint(1, 50)
                       },
                       random_search=True,
                       n_iter=250,
                       scoring='roc_auc',
                       verbose=True)
rf_rocs = test_model(rf_model,
                     x, y,
                     allow_indeterminates=True)

## Sklearn Ensemble Model

In [ ]:
# Grid search params
clf1 = svm.SVC(probability=True)
clf2 = linear_model.LogisticRegression()

eclf = ensemble.VotingClassifier(
    estimators=[('svm', clf1), ('lr', clf2)],
    voting='soft')

params = {
    'svm__C': np.logspace(-3, 2, 100),
    'svm__gamma': np.logspace(-3, 2, 100),
    'svm__kernel': ['linear', 'rbf', 'poly'],
    'lr__C': np.logspace(-3, 2, 100)
}

# Test model! 5-fold CV with hyperparameter optimization
clf = ScikitModel(
    eclf,
    params,
    random_search=True, 
    n_iter=100, 
    verbose=True)

ensemble_rocs = test_model(clf, x, y, allow_indeterminates=True)

In [ ]:
plot_cv_roc_curves(ensemble_rocs)